In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
np.random.seed(2024)
print("data preprocess start")
print("*"*30)

raw_interaction = pd.read_csv('/home/zyx2509/Food_dataset/Foodcom/raw_dataset/RAW_interactions.csv')
pp_recipe = pd.read_csv('/home/zyx2509/Food_dataset/Foodcom/raw_dataset/PP_recipes.csv')

records_removed = True
df = raw_interaction[raw_interaction['recipe_id'].isin(pp_recipe['id'].tolist())]

while records_removed:
    # 统计每个用户和物品的出现次数
    user_counts = df['user_id'].value_counts()
    item_counts = df['recipe_id'].value_counts()

    # 筛选出现次数至少为5次的用户和物品
    valid_users = user_counts[user_counts >= 5].index
    valid_items = item_counts[item_counts >= 5].index

    # 使用筛选后的用户和物品过滤原始数据框
    filtered_df = df[df['user_id'].isin(valid_users) & df['recipe_id'].isin(valid_items)]

    # 检查是否有记录被删除
    if len(filtered_df) == len(df):
        records_removed = False
    else:
        records_removed = True

    # 更新原始数据框
    df = filtered_df
core_inter = df
sort_filter_ui = core_inter.sort_values('date').reset_index(drop=True)

sort_filter_ui

data preprocess start
******************************


,user_id,recipe_id,date,rating,review
0,2312,780,2000-09-12,5,This was delicious.\r\n
1,2999,3567,2000-10-23,5,I have made this pie instead of plain ol' pump...
2,2178,3704,2000-10-30,3,Careful not to cook it too long... you want th...
3,2178,4366,2000-11-04,5,"if you like oysters, this is a great alternati..."
4,3794,191,2000-12-15,5,BOY! You sure can't eat just one of these muf...
...,...,...,...,...,...
464732,2001513060,367414,2018-12-17,1,"maybe I did something wrong , but I thought th..."
464733,2001513060,192495,2018-12-17,5,This is a keeper. Delicious Soup that both my ...
464734,454804,20713,2018-12-17,0,"Made this as gifts. Did 6, quart jars plus had..."
464735,1290903,131607,2018-12-18,5,This is a great recipe for a nice thin crispy ...


In [2]:
import os
Dir = r'/home/zyx2509/Food_dataset/Foodcom/'
image_dir = 'image_dataset'
dst_dir = Dir + image_dir

file_names = os.listdir(dst_dir)

finish_image = [int(os.path.splitext(file_name)[0]) for file_name in file_names]
sort_filter_ui = sort_filter_ui[sort_filter_ui['recipe_id'].isin(finish_image)]
print(len(finish_image))
sort_filter_ui

29944


,user_id,recipe_id,date,rating,review
1,2999,3567,2000-10-23,5,I have made this pie instead of plain ol' pump...
2,2178,3704,2000-10-30,3,Careful not to cook it too long... you want th...
3,2178,4366,2000-11-04,5,"if you like oysters, this is a great alternati..."
6,5523,7695,2001-02-01,1,I agree.
9,42189,4460,2001-02-12,5,I have had this before. It has really good fla...
...,...,...,...,...,...
464732,2001513060,367414,2018-12-17,1,"maybe I did something wrong , but I thought th..."
464733,2001513060,192495,2018-12-17,5,This is a keeper. Delicious Soup that both my ...
464734,454804,20713,2018-12-17,0,"Made this as gifts. Did 6, quart jars plus had..."
464735,1290903,131607,2018-12-18,5,This is a great recipe for a nice thin crispy ...


In [3]:

train_ = sort_filter_ui[:int(0.6*len(sort_filter_ui))]
valid_ = sort_filter_ui[int(0.6*len(sort_filter_ui)):int(0.7*len(sort_filter_ui))]
test_ = sort_filter_ui[int(0.7*len(sort_filter_ui)):]
print(len(train_), len(valid_), len(test_))
u_tr = set(train_['user_id'].tolist())
u_te = set(test_['user_id'].tolist())
u_total = u_tr & u_te
print(len(u_tr), len(u_te), len(u_total))
filter_u_tr = train_[train_['user_id'].isin(u_total)]
filter_u_te = test_[test_['user_id'].isin(u_total)]
filter_u_va = valid_[valid_['user_id'].isin(u_total)]
print(len(filter_u_tr), len(filter_u_va), len(filter_u_te))
u_va = set(filter_u_va['user_id'].tolist())
i_tr = set(filter_u_tr['recipe_id'].tolist())
i_te = set(filter_u_te['recipe_id'].tolist())
i_va = set(filter_u_va['recipe_id'].tolist())
i_total = i_tr|i_va|i_te
i_cold = (i_va|i_te)-i_tr
print(len(u_va))
print(len(i_tr), len(i_va), len(i_te))
print(len(i_total), len(i_cold))
print(len(i_va - i_tr), len(i_te-i_tr))

257107 42852 128554
12764 10557 7596
191893 35163 96143
4821
25832 14229 23029
29943 4111
1711 4000


In [4]:
from sklearn.preprocessing import LabelEncoder

user_ids = list(u_total)
user_encoder = LabelEncoder().fit(user_ids)

user_to_idx = {v: i for i, v in enumerate(user_encoder.classes_)}
idx_to_user = {i: v for i, v in enumerate(user_encoder.classes_)}

item_ids = list(i_total)
item_encoder = LabelEncoder().fit(item_ids)

item_to_idx = {v: i for i, v in enumerate(item_encoder.classes_)}
idx_to_item = {i: v for i, v in enumerate(item_encoder.classes_)}

In [5]:

train_interaction = filter_u_tr

test_interaction = filter_u_te

valid_interaction = filter_u_va

print(len(train_interaction))

191893


In [6]:
file_path = '/home/zyx2509/Food_dataset/Foodcom/'

train_interaction['u'] = train_interaction['user_id'].map(user_to_idx)
train_interaction['i'] = train_interaction['recipe_id'].map(item_to_idx)
train_sorted = train_interaction.sort_values(by=['u', 'i'])
train_to_save = train_sorted[['u', 'i', 'rating']]
train_to_save['rating'] = train_to_save['rating'].astype(int)
train_to_save.to_csv(file_path+'processed_dataset/data.train.rating', sep='	', header=False, index=False)

test_interaction['u'] = test_interaction['user_id'].map(user_to_idx)
test_interaction['i'] = test_interaction['recipe_id'].map(item_to_idx)
test_sorted = test_interaction.sort_values(by=['u', 'i'])
test_to_save = test_sorted[['u', 'i', 'rating']]
test_to_save['rating'] = test_to_save['rating'].astype(int)
test_to_save.to_csv(file_path+'processed_dataset/data.test.rating', sep='	', header=False, index=False)

valid_interaction['u'] = valid_interaction['user_id'].map(user_to_idx)
valid_interaction['i'] = valid_interaction['recipe_id'].map(item_to_idx)
valid_sorted = valid_interaction.sort_values(by=['u', 'i'])
valid_to_save = valid_sorted[['u', 'i', 'rating']]
valid_to_save['rating'] = valid_to_save['rating'].astype(int)
valid_to_save.to_csv(file_path+'processed_dataset/data.valid.rating', sep='	', header=False, index=False)

/tmp/ipykernel_1740231/583291010.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_interaction['u'] = train_interaction['user_id'].map(user_to_idx)
/tmp/ipykernel_1740231/583291010.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_interaction['i'] = train_interaction['recipe_id'].map(item_to_idx)
/tmp/ipykernel_1740231/583291010.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [7]:
import pandas as pd
import pickle

train_df = pd.read_csv('/home/zyx2509/Food_dataset/Foodcom/processed_dataset/data.train.rating', names=['u', 'i', 'r'], sep='\t')
test_df = pd.read_csv('/home/zyx2509/Food_dataset/Foodcom/processed_dataset/data.test.rating', names=['u', 'i', 'r'], sep='\t')

test_i = test_df['i'].unique()
train_i = train_df['i'].unique()

cold = set(test_i) - set(train_i)
print(len(cold), len(test_i), len(train_i))

4000 23029 25832


In [8]:
# algo 1
from collections import Counter, defaultdict
from tqdm import tqdm

ingre_num_df = pp_recipe[['id','ingredient_ids']]
ingre_num_df = ingre_num_df[ingre_num_df['id'].isin(item_to_idx.keys())]
print(len(ingre_num_df))

ingre_num_df['i'] = ingre_num_df['id'].map(item_to_idx)
ingre_num_df['ingredient_list'] = ingre_num_df['ingredient_ids'].apply(lambda x: eval(x))

ingre_num_df['i'] = ingre_num_df['id'].map(item_to_idx)
ingre_num_df = ingre_num_df.sort_values(by=['i'])
print(ingre_num_df)
train_ingre_df = ingre_num_df[ingre_num_df['i'].isin(train_i)]

df_ing_list = train_ingre_df['ingredient_list'].tolist()
all_ingre_list = [item for sublist in df_ing_list for item in sublist]
frequency_count = Counter(all_ingre_list)
filtered_ingre = set([key for key, value in frequency_count.items() if value >= 1])

ingre_num_df['ingredient_del'] = ingre_num_df['ingredient_list'].apply(
    lambda x: [i for i in x if i in filtered_ingre])
ingre_num_df['ingre_num'] = ingre_num_df['ingredient_del'].apply(lambda x: len(x))

print(ingre_num_df['ingre_num'].value_counts())
print(len(filtered_ingre))

new_ingre = []
for index, row in tqdm(ingre_num_df.iterrows()):
    if row['ingre_num']==0:
        new_ingre.extend(row['ingredient_list'])
    else:
        new_ingre.extend(row['ingredient_del']) 

final_ingre = set(new_ingre)
ingre_num_df['new_ingres'] = ingre_num_df['ingredient_list'].apply(
    lambda x: [i for i in x if i in final_ingre])
ingre_num_df['new_ingres_num'] = ingre_num_df['new_ingres'].apply(lambda x: len(x))

print(len(final_ingre))
print(ingre_num_df['new_ingres_num'].min(), ingre_num_df['new_ingres_num'].max(), ingre_num_df['new_ingres_num'].mean())

29943
            id                                     ingredient_ids      i  \
52588       40               [6906, 4303, 2919, 2832, 3829, 1567]      0   
26668       49  [2847, 840, 1254, 2683, 4593, 1257, 6270, 4836...      1   
101951      58  [451, 7557, 2379, 6270, 5319, 437, 836, 3184, ...      2   
110880      66  [739, 4053, 6009, 2045, 5010, 3203, 2066, 3783...      3   
84384      142  [6906, 4574, 2499, 106, 7470, 63, 7407, 335, 6...      4   
...        ...                                                ...    ...   
141784  522871                [756, 7655, 7654, 6696, 6248, 6906]  29938   
126326  522889          [3559, 2499, 4333, 4883, 6270, 1118, 840]  29939   
88162   532736                           [7474, 6889, 7702, 5695]  29940   
170005  532740                     [7998, 5476, 7642, 3723, 3497]  29941   
77830   536119  [1647, 5915, 6335, 4493, 6413, 5434, 6696, 608...  29942   

                                          ingredient_list  
52588                

29943it [00:01, 25230.47it/s]


4963
3 20 8.818121096750492


In [9]:

from sklearn.preprocessing import LabelEncoder

save_ingre_num = ingre_num_df[['i', 'new_ingres_num']]
save_ingre_num.to_csv(file_path + 'processed_dataset/data_id_ingre_num_file', sep='	', header=False, index=False)

ingre_ids = list(final_ingre)
ingre_encoder = LabelEncoder().fit(ingre_ids)

ingre_to_idx = {v: i for i, v in enumerate(ingre_encoder.classes_)}
idx_to_ingre = {i: v for i, v in enumerate(ingre_encoder.classes_)}
ingre_num_df['ingredient_idx'] = ingre_num_df['new_ingres'].apply(lambda x: [ingre_to_idx[i] for i in x])

dict_data = dict(zip(ingre_num_df['i'], ingre_num_df['ingredient_idx'].tolist()))

fill_value = len(ingre_ids)
ingre_code = np.full((len(dict_data), ingre_num_df['new_ingres_num'].max()), fill_value)

for i, (key, value) in enumerate(dict_data.items()):
    ingre_code[key, :len(value)] = value

# save id ingre_code file
# total code num = 33147 
np.save(file_path + 'processed_dataset/data_ingre_code_file.npy', ingre_code)

print(ingre_code.shape, np.max(ingre_code))
print(ingre_code, np.max(ingre_code), np.min(ingre_code))
ri_list = []
for i, sublist in tqdm(enumerate(ingre_code)):
    for j in sublist:
        if j != np.max(ingre_code):
            ri_list.append([i, j])
print(len(ri_list))
ri_array = np.array(ri_list).astype(np.int_)
print(ri_array.shape)

ri_path = file_path + 'processed_dataset/ri_graph.txt'
with open(ri_path, 'w') as f:
    np.savetxt(f, ri_array, fmt='%d')
print(f"NumPy array saved as {ri_path}")

(29943, 20) 4963
[[4277 2698 1845 ... 4963 4963 4963]
 [1789  510  776 ... 4963 4963 4963]
 [ 274 4681 1495 ... 4963 4963 4963]
 ...
 [4641 4265 4762 ... 4963 4963 4963]
 [4951 3427 4732 ... 4963 4963 4963]
 [1024 3678 3941 ... 4963 4963 4963]] 4963 0


29943it [01:51, 268.80it/s]


264041
(264041, 2)
NumPy array saved as /home/zyx2509/Food_dataset/Foodcom/processed_dataset/ri_graph.txt


In [ ]:
from tqdm import tqdm
import pickle
import torch

# 打开.pkl文件并读取数据
with open(file_path+'raw_dataset/ingr_map.pkl', 'rb') as f:
    data = pickle.load(f)

# 使用读取的数据
from transformers import T5Tokenizer, T5EncoderModel

# 加载T5模型和tokenizer
model_name = '/home/zyx2509/CIKM_code/t5-small'  # 可以选择其他T5变体和大小
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5EncoderModel.from_pretrained(model_name).cuda()

def t5_encoder(row):
    inputs = tokenizer(row, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: value.cuda() for key, value in inputs.items()}

    # 生成表示
    with torch.no_grad():
        outputs = model(**inputs)
    model_output = outputs.last_hidden_state
    t5_out = torch.mean(model_output, dim=1, keepdim=False).squeeze().detach().cpu().numpy()

    return t5_out
    # 存储特征向量

data['t5_out'] = data['processed'].apply(lambda x: t5_encoder(x))

print(data)

In [ ]:
def calculate_mean(arrays):
    # 使用 np.mean 计算 NumPy 数组的平均值
    return np.mean(arrays, axis=0)
result_df = data.groupby('id')['t5_out'].apply(calculate_mean).reset_index()
result_df.columns = ['id', 'MeanArray']
result_df = result_df[result_df['id'].isin(ingre_ids)]
result_df['idx'] = result_df['id'].apply(lambda x:ingre_to_idx[x])
result_df = result_df.sort_values(['idx'])
print(result_df)

In [ ]:
ingre_feature = np.array(result_df['MeanArray'].tolist())
# 保存数组到.npy文件中
np.save(file_path+'processed_dataset/data_ingre_features_t5.npy', ingre_feature)


In [ ]:

raw_recipe = pd.read_csv('/home/zyx2509/Food_dataset/Foodcom/raw_dataset/RAW_recipes.csv')

texture_df = raw_recipe[raw_recipe['id'].isin(i_total)]

texture_df = texture_df[['id', 'name', 'steps', 'description']]
texture_df['i'] = texture_df['id'].map(item_to_idx)
texture_df = texture_df.sort_values(by=['i'])

title_dict = defaultdict(list)
direction_dict = defaultdict(list)
description_dict = defaultdict(list)

for idx, row in tqdm(texture_df.iterrows()):
    recipe_id = row['i']
    title_dict[recipe_id] += [row['name']]
    direction = eval(row['steps'])
    direction_dict[recipe_id] += direction
    description_dict[recipe_id] += [row['description']] 



In [ ]:

# 存储文本向量的列表
title_vectors = {}

# 遍历图片文件夹
for key, value in tqdm(title_dict.items()):
    inputs = tokenizer(value, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: value.cuda() for key, value in inputs.items()}

    # 生成表示
    with torch.no_grad():
        outputs = model(**inputs)
    model_output = outputs.last_hidden_state

    # 存储特征向量
    title_vectors[key] = torch.mean(model_output, dim=1, keepdim=False).squeeze().detach().cpu().numpy()

print(len(title_vectors))

title_feature = np.zeros((len(title_vectors), title_vectors[0].shape[0]))

for i, (key, value) in enumerate(title_vectors.items()):
    title_feature[key] = value

print(title_feature.shape)
np.save(file_path + 'processed_dataset/data_title_features_t5.npy', title_feature)
print(f"save title feature:{title_feature.shape}")


In [ ]:

text_feature = np.zeros((len(ingre_code), ingre_feature.shape[1]))
for i, ingre_list in enumerate(ingre_code):
    mask = ingre_list != len(ingre_to_idx)
    new_ingre = ingre_list[mask]
    ingre_embs = ingre_feature[new_ingre]
    ingre_embs = np.concatenate((ingre_embs, np.expand_dims(title_feature[i], axis=0)), axis=0)
    text_feature[i] = ingre_embs.mean(0)
print(text_feature.shape)
np.save(file_path + 'processed_dataset/data_text_features_t5.npy', text_feature)

In [ ]:

filename = file_path + 'processed_dataset/mapping_dict.pkl'

# 将字典保存到 pkl 文件中
with open(filename, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump((user_to_idx, item_to_idx, ingre_to_idx), f)
print("save mapping dict")
print(len(ingre_to_idx), len(user_to_idx), len(item_to_idx))

In [11]:
import torch
from torchvision import models, transforms
from PIL import Image
import os

# 加载预训练的ResNet-50模型
resnet = models.resnet50(pretrained=True)
# 将模型设置为评估模式
resnet.fc = torch.nn.Identity()
resnet.cuda()
resnet.eval()

# 图片文件夹路径
image_folder = file_path+'image_dataset/'

# 数据预处理
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 存储图片向量的列表
image_vectors = {}

# 遍历图片文件夹
num=0
for index, row in tqdm(ingre_num_df.iterrows()):
    image_id = row['id']
    image_path = os.path.join(image_folder, f"{image_id}.jpg")
    # 加载并预处理图片
    image = Image.open(image_path)


    try:
        image = preprocess(image)
        # 增加一个维度，以符合模型的输入要求
        image = image.unsqueeze(0).cuda()
        # 使用模型提取特征
        with torch.no_grad():
            output = resnet(image)
        # 提取特征向量，并转换为numpy数组
    except Exception as e:
        print(e, image_path)
        image = image.convert('RGB')
        image = preprocess(image)
        # 增加一个维度，以符合模型的输入要求
        image = image.unsqueeze(0).cuda()
        # 使用模型提取特征
        with torch.no_grad():
            output = resnet(image)

    vector = output.squeeze().detach().cpu().numpy()
    # 存储特征向量
    image_vectors[image_id] = vector

print(len(image_vectors))

29943it [10:43, 46.53it/s] 

29943


In [12]:
convert_image_vectors = {item_to_idx[key]: value for key, value in image_vectors.items()}

sorted_convert_image_vectors = dict(sorted(convert_image_vectors.items()))
print(sorted_convert_image_vectors[0].shape)

image_feature = np.zeros((len(sorted_convert_image_vectors), sorted_convert_image_vectors[0].size))

# 遍历字典，将值填充到数组中
for i, (key, value) in enumerate(sorted_convert_image_vectors.items()):
        image_feature[key] = value

print(image_feature.shape)
np.save(file_path+'processed_dataset/data_image_features_float.npy', image_feature)

(2048,)
(29943, 2048)


In [14]:
file_path = '/home/zyx2509/Food_dataset/Foodcom/processed_dataset/'
data_file = file_path+'data.train.rating'
test_file = file_path+'data.test.rating'
valid_file = file_path+'data.valid.rating'

train_df = pd.read_csv(data_file, names=['user', 'item', 'rate'], sep='\t')
test_df = pd.read_csv(test_file, names=['user', 'item', 'rate'], sep='\t')
valid_df = pd.read_csv(valid_file, names=['user', 'item', 'rate'], sep='\t')

from collections import defaultdict

item_count = defaultdict(int)
user_items = {}

# 遍历 DataFrame 中的每一行
for index, row in tqdm(train_df.iterrows()):
        # 提取第一列的值作为聚类的键
        u = int(row['user'])
        # 如果键不存在于字典中，则将其初始化为一个空列表
        if u not in user_items:
            user_items[u] = []
        # 将第二列的值添加到对应键的列表中
        user_items[u].append(int(row['item']))
        item_count[int(row['item'])] += 1

print(len(item_count)) 

all_item = list(item_count.keys())
count = list(item_count.values())
sum_value = np.sum([x for x in count])
popularity = [value / sum_value for value in count]

sum_prob = np.sum([x**0.7 for x in popularity])
probability = [value**0.7/sum_prob for value in popularity]

np.random.seed(2024)
item_set = list(item_count.keys())
test_num = 500
sample_type = 'popular bias'
test_neg_items = defaultdict()
valid_neg_items = defaultdict()
valid_user_set = set(valid_df['user'])
test_user_set = set(test_df['user'])

for user, user_seq in tqdm(user_items.items()):
    if user not in test_user_set:
        continue
    test_samples = []
    while len(test_samples) < test_num:
        sample_ids = np.random.choice(all_item, 2*test_num, replace=False, p=probability)
        sample_ids_set = set(sample_ids)
        user_seq_set = set(user_seq)
        result_set = sample_ids_set - user_seq_set
        test_samples = list(result_set)
    test_samples = test_samples[:test_num]
    test_neg_items[user] = test_samples

for user, user_seq in tqdm(user_items.items()):
    if user not in valid_user_set:
        continue
    valid_samples = []
    while len(valid_samples) < test_num:
        sample_ids = np.random.choice(all_item, 2*test_num, replace=False, p=probability)
        sample_ids_set = set(sample_ids)
        user_seq_set = set(user_seq)
        result_set = sample_ids_set - user_seq_set
        valid_samples = list(result_set)
    valid_samples = valid_samples[:test_num]
    valid_neg_items[user] = valid_samples

191893it [00:07, 25196.31it/s]


25832


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7596/7596 [00:22<00:00, 342.47it/s]


In [15]:
test_pos_dict = {}
valid_pos_dict = {}

for index, row in tqdm(valid_df.iterrows()):
        # 提取第一列的值作为聚类的键
        u = int(row['user'])
        if u not in valid_pos_dict:
            valid_pos_dict[u] = []
        valid_pos_dict[u].append(int(row['item']))

for index, row in tqdm(test_df.iterrows()):
        # 提取第一列的值作为聚类的键
        u = int(row['user'])
        if u not in test_pos_dict:
            test_pos_dict[u] = []
        test_pos_dict[u].append(int(row['item']))
        
test_sample_pos = pd.DataFrame({'KeyValue': [f"{k}:{v}" for k, v in test_pos_dict.items()]}, index=test_pos_dict.keys())
valid_sample_pos = pd.DataFrame({'KeyValue': [f"{k}:{v}" for k, v in valid_pos_dict.items()]}, index=valid_pos_dict.keys())

print(test_sample_pos[:5])
print(valid_sample_pos[:5])

35163it [00:01, 25082.22it/s]
96143it [00:03, 26737.48it/s]


                                            KeyValue
0  0:[590, 632, 1326, 1691, 2121, 2300, 2745, 284...
1                                          1:[24125]
2                                     2:[2374, 5285]
3                                           3:[3172]
4                            4:[10170, 18628, 23178]
                                            KeyValue
0  0:[76, 3740, 6171, 6584, 6867, 6886, 8996, 152...
2                                      2:[57, 10237]
4                                           4:[3090]
7                             7:[2432, 16061, 24274]
9  9:[10735, 11113, 12140, 12311, 13840, 19339, 1...


In [16]:

test_sample_neg = pd.DataFrame.from_dict(test_neg_items).transpose()
valid_sample_neg = pd.DataFrame.from_dict(valid_neg_items).transpose()

print(len(test_sample_neg))
print(len(valid_sample_neg))


7596
4821


In [17]:
test_sample_concat = pd.concat([test_sample_pos, test_sample_neg], axis=1)
print(len(test_sample_concat))

valid_sample_concat = pd.concat([valid_sample_pos, valid_sample_neg], axis=1)
print(len(valid_sample_concat))

7596
4821


In [18]:
test_sample_concat['KeyValue'] = test_sample_concat['KeyValue'].apply(lambda x: x.replace('[', '').replace(']', '').replace(' ', ''))
test_sample_concat['KeyValue'] = test_sample_concat['KeyValue'].apply(lambda x: f"({x})")

print(test_sample_concat.iloc[:5, :5])

valid_sample_concat['KeyValue'] = valid_sample_concat['KeyValue'].apply(lambda x: x.replace('[', '').replace(']', '').replace(' ', ''))
valid_sample_concat['KeyValue'] = valid_sample_concat['KeyValue'].apply(lambda x: f"({x})")

print(valid_sample_concat.iloc[:5, :5])

                                            KeyValue      0      1      2  \
0  (0:590,632,1326,1691,2121,2300,2745,2849,3197,...  16385   6146      4   
1                                          (1:24125)  22528   4097  12291   
2                                      (2:2374,5285)   4101  20485   6159   
3                                           (3:3172)   6145  22532   6149   
4                              (4:10170,18628,23178)   8192   2050  16386   

       3  
0  18444  
1  24582  
2  12305  
3  14343  
4   4101  
                                            KeyValue      0      1      2  \
0  (0:76,3740,6171,6584,6867,6886,8996,15281,1792...   8192  18441   6156   
2                                       (2:57,10237)  14336   6144   4101   
4                                           (4:3090)  12291  16389   4101   
7                               (7:2432,16061,24274)  14336  12288  10245   
9      (9:10735,11113,12140,12311,13840,19339,19764)  20480   2053  12298   

       3

In [19]:
valid_sample_concat.to_csv(file_path+'data.valid.negative', sep='	', header=False, index=False)
test_sample_concat.to_csv(file_path+'data.test.negative', sep='	', header=False, index=False)


In [20]:
from scipy.sparse import coo_matrix

src = train_to_save['u'].values
tgt = train_to_save['i'].values
data = np.ones(len(train_to_save))
mat = coo_matrix((data, (src, tgt)), shape=(len(user_to_idx), len(item_to_idx)))

# 指定保存到的文件名
import pickle
filename = file_path+'inter_coo_matrix.pkl'

# 将字典保存到 pkl 文件中
with open(filename, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump(mat, f)

In [21]:
# now, begin process graph edges
# 1 user-recipe graph edge
import numpy as np

inter_mat = []
lines = open(file_path + 'data.train.rating', 'r').readlines()
for line in lines:
    tokens = line.strip().split('\t')
    u_id, pos_id = int(tokens[0]), int(tokens[1])
    inter_mat.append([u_id, pos_id])

ur_array = np.array(inter_mat).astype(np.int_)

print(ur_array.shape)

ur_path = file_path + 'graph_edge/ur_graph.txt'
with open(ur_path, 'w') as f:
    np.savetxt(f, ur_array, fmt='%d')
print("NumPy array saved as 'ur_graph.txt'")

with open(ur_path, 'r') as f:
    ab = np.loadtxt(f, dtype=np.int_)
print(f"testing for shape, ur_graph is {ab.shape}")


(191893, 2)
NumPy array saved as 'ur_graph.txt'
testing for shape, ur_graph is (191893, 2)


In [22]:
# 2 recipe-recipe graph edge 
# 2 pending
# 3 recipe-ingredient graph edge
from tqdm import tqdm
ingre_code = np.load(file_path + 'data_ingre_code_file.npy')
print(ingre_code, np.max(ingre_code), np.min(ingre_code))
ri_list = []
for i, sublist in tqdm(enumerate(ingre_code)):
    for j in sublist:
        if j!=np.max(ingre_code):
            ri_list.append([i, j])
print(len(ri_list))
ri_array = np.array(ri_list).astype(np.int_)
print(ri_array.shape)

ri_path = file_path + 'graph_edge/ri_graph.txt'
with open(ri_path, 'w') as f:
    np.savetxt(f, ri_array, fmt='%d')
print("NumPy array saved as 'ri_graph.txt'")

with open(ri_path, 'r') as f:
    ab = np.loadtxt(f, dtype=np.int_)
print(f"testing for shape, ri_graph is {ab.shape}")

[[4277 2698 1845 ... 4963 4963 4963]
 [1789  510  776 ... 4963 4963 4963]
 [ 274 4681 1495 ... 4963 4963 4963]
 ...
 [4641 4265 4762 ... 4963 4963 4963]
 [4951 3427 4732 ... 4963 4963 4963]
 [1024 3678 3941 ... 4963 4963 4963]] 4963 0


29943it [01:42, 292.88it/s]


264041
(264041, 2)
NumPy array saved as 'ri_graph.txt'
testing for shape, ri_graph is (264041, 2)


In [24]:
# 4 ingredient-ingredient graph edge
# 4 used for fgcn model
from collections import Counter, defaultdict
with open('/home/zyx2509/Food_dataset/Foodcom/raw_dataset/ingr_map.pkl', 'rb') as f:
    data = pickle.load(f)

mapping_dict = file_path + '/mapping_dict.pkl'

with open(mapping_dict, 'rb') as f:
    u_to_idx, i_to_idx, ingre_to_idx = pickle.load(f)

ingreText_to_idx = {}
for key, value in tqdm(ingre_to_idx.items()):
    result = ''
    rows_data = data[data['id']==key]
    for index, row in rows_data.iterrows():
        result += row['processed']
    ingreText_to_idx[result] = value

color_set = ['white', 'black', 'red', 'green', 'yellow']
ingre_with_colors = defaultdict(list)

# 遍历字典中的每个值
for key, value in tqdm(ingreText_to_idx.items()):
    # 检查值中是否包含颜色列表中的任何一个颜色
    for color in color_set:
        if color in key:
            # 如果包含颜色，则将对应的键保存到列表中
            ingre_with_colors[color].append(value)

shape_set = ['slice', 'dice', 'minced', 'powder', 'roll', 'shred']
ingre_with_shapes = defaultdict(list)

# 遍历字典中的每个值
for key, value in tqdm(ingreText_to_idx.items()):
    # 检查值中是否包含颜色列表中的任何一个颜色
    for shape in shape_set:
        if shape in key:
            # 如果包含颜色，则将对应的键保存到列表中
            ingre_with_shapes[shape].append(value)

cooking_set = ['deep-fry', 'dry', 'fry', 'steam', 'boil', 'pickle']
ingre_with_cookings = defaultdict(list)

# 遍历字典中的每个值
for key, value in tqdm(ingreText_to_idx.items()):
    # 检查值中是否包含颜色列表中的任何一个颜色
    for cooking in cooking_set:
        if cooking in key:
            # 如果包含颜色，则将对应的键保存到列表中
            ingre_with_cookings[cooking].append(value)

edge_dict = defaultdict(int)

for key, lst in tqdm(ingre_with_colors.items()):
    for i in range(len(lst)):
        for j in range(i + 1, len(lst)):
            edge_dict[(lst[i],lst[j])] += 1
for key, lst in tqdm(ingre_with_shapes.items()):
    for i in range(len(lst)):
        for j in range(i + 1, len(lst)):
            edge_dict[(lst[i],lst[j])] += 1
for key, lst in tqdm(ingre_with_cookings.items()):
    for i in range(len(lst)):
        for j in range(i + 1, len(lst)):
            edge_dict[(lst[i],lst[j])] += 1
print(len(edge_dict))
value_counts = Counter(val for val in edge_dict.values())

# 打印结果
print("每个元素出现的次数：", value_counts)

new_edge_list = []
for edge_pair, count in tqdm(edge_dict.items()):
    new_edge_list.append([edge_pair[0], edge_pair[1]])
    new_edge_list.append([edge_pair[1], edge_pair[0]])

print(len(new_edge_list))
print(new_edge_list[:10])

ii_array = np.array(new_edge_list).astype(np.int_)

print(ii_array.shape)

ii_path = file_path + 'graph_edge/ii_graph.txt'
with open(ii_path, 'w') as f:
    np.savetxt(f, ii_array, fmt='%d')
print(f"NumPy array saved as {ii_path}")

with open(ii_path, 'r') as f:
    ab = np.loadtxt(f, dtype=np.int_)
print(ab.shape)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 5485.62it/s]


38235
每个元素出现的次数： Counter({1: 37810, 2: 423, 3: 2})


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38235/38235 [00:00<00:00, 73990.87it/s]


76470
[[26, 27], [27, 26], [26, 28], [28, 26], [26, 172], [172, 26], [26, 341], [341, 26], [26, 457], [457, 26]]
(76470, 2)
NumPy array saved as /home/zyx2509/Food_dataset/Foodcom/processed_dataset/graph_edge/ii_graph.txt
(76470, 2)


In [25]:
# 5 recipe-calories graph edge
# 7 calories level dict
# 5 and 7 used for schgn model
# build calories dict for 5 and 7
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder

calories_dict = {}
raw_recipe = pd.read_csv('/home/zyx2509/Food_dataset/Foodcom/raw_dataset/RAW_recipes.csv')
with open(file_path+'mapping_dict.pkl', 'rb') as f:
    _, item_to_idx, _ = pickle.load(f)

items = item_to_idx.keys()
print(len(items))
now_recipe = raw_recipe[raw_recipe['id'].isin(items)]
now_recipe['cal'] = now_recipe['nutrition'].apply(lambda x: int(eval(x)[0]/50))
now_recipe['i'] = now_recipe['id'].apply(lambda x: item_to_idx[x])
print(now_recipe['cal'].max(), now_recipe['cal'].min(), now_recipe['cal'].mean())
print((now_recipe['cal'] == 0).sum())

cal_encoder = LabelEncoder().fit(list(now_recipe['cal'].unique()))
cal_to_idx = {v: i for i, v in enumerate(cal_encoder.classes_)}

now_recipe['cal_norm'] = now_recipe['cal'].apply(lambda x: cal_to_idx[x])
now_recipe = now_recipe.sort_values(by='i')

for index, row in now_recipe.iterrows():
    recipe_id = row['i']
    calories_amount = row['cal_norm']
    calories_dict[recipe_id] = calories_amount
# 5 recipe-calories graph edge
rc_list = []
for key, value in calories_dict.items():
    rc_list.append([key, value])
rc_array = np.array(rc_list).astype(np.int_)
print(rc_array.shape)

rc_path = file_path + 'graph_edge/rc_graph.txt'
with open(rc_path, 'w') as f:
    np.savetxt(f, rc_array, fmt='%d')
print("NumPy array saved as 'rc_graph.txt'")

with open(rc_path, 'r') as f:
    ab = np.loadtxt(f, dtype=np.int_)
print(f"testing for shape, rc_graph is {ab.shape}")

# 7 calories level dict
rc_dict = file_path + 'graph_edge/recipe_cal_level_dict.pkl'

# 将字典保存到 pkl 文件中
with open(rc_dict, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump(calories_dict, f)
print("save calories dict")

cal_to_idx_map_file = file_path + 'graph_edge/recipe_cal_level_map.pkl'
with open(cal_to_idx_map_file, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump(cal_to_idx, f)
print("save cal_to_idx dict")


29943


/tmp/ipykernel_1740231/3030144398.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  now_recipe['cal'] = now_recipe['nutrition'].apply(lambda x: int(eval(x)[0]/50))
/tmp/ipykernel_1740231/3030144398.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  now_recipe['i'] = now_recipe['id'].apply(lambda x: item_to_idx[x])
/tmp/ipykernel_1740231/3030144398.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

773 0 8.005844437765086
983
(29943, 2)
NumPy array saved as 'rc_graph.txt'
testing for shape, rc_graph is (29943, 2)
save calories dict
save cal_to_idx dict


In [26]:
# begin to calculate health level
import pandas as pd
import pickle
from tqdm import tqdm


# 将字典保存到 pkl 文件中
raw_recipe = pd.read_csv('/home/zyx2509/Food_dataset/Foodcom/raw_dataset/RAW_recipes.csv')
with open(file_path+'mapping_dict.pkl', 'rb') as f:
    _, item_to_idx, _ = pickle.load(f)
dataset_recipe = raw_recipe[raw_recipe['id'].isin(list(item_to_idx.keys()))]
recipe_nutrition = dataset_recipe[['id', 'nutrition']]
recipe_nutrition['i'] = recipe_nutrition['id'].apply(lambda x: item_to_idx[x])
recipe_nutrition['cal'] = recipe_nutrition['nutrition'].apply(lambda x: eval(x)[0])
recipe_nutrition['fat'] = recipe_nutrition['nutrition'].apply(lambda x: eval(x)[1])
recipe_nutrition['sugar'] = recipe_nutrition['nutrition'].apply(lambda x: eval(x)[2])
recipe_nutrition['sodium'] = recipe_nutrition['nutrition'].apply(lambda x: eval(x)[3])
recipe_nutrition['protein'] = recipe_nutrition['nutrition'].apply(lambda x: eval(x)[4])
recipe_nutrition['saturated_fat'] = recipe_nutrition['nutrition'].apply(lambda x: eval(x)[5])
recipe_nutrition['carbohydrates'] = recipe_nutrition['nutrition'].apply(lambda x: eval(x)[6])

for idx, row in tqdm(recipe_nutrition.iterrows()):
    health_level = 0
    # 65g
    if 15<=row['fat']<=30:
        health_level+=1
    # 50g
    if row['sugar']<10:
        health_level+=1
    # WHO healthy sodium 2g per day, daily 2400mg, 2/2.4=83
    # WHO healthy salt 5g per day, daily 6000mg
    if row['sodium']<83:
        health_level+=1
    # 50g
    if 10<=row['protein']<=15:
        health_level+=1
    # 20g
    if row['saturated_fat']<10:
        health_level+=1 
    # 300g
    if 55<=row['carbohydrates']<=75:
        health_level+=1
    recipe_nutrition.at[idx, 'health'] = health_level
recipe_nutrition

/tmp/ipykernel_1740231/101115609.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe_nutrition['i'] = recipe_nutrition['id'].apply(lambda x: item_to_idx[x])
/tmp/ipykernel_1740231/101115609.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe_nutrition['cal'] = recipe_nutrition['nutrition'].apply(lambda x: eval(x)[0])
/tmp/ipykernel_1740231/101115609.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,id,nutrition,i,cal,fat,sugar,sodium,protein,saturated_fat,carbohydrates,health
15,63986,"[105.7, 8.0, 0.0, 26.0, 5.0, 4.0, 3.0]",6678,105.7,8.0,0.0,26.0,5.0,4.0,3.0,3.0
16,43026,"[94.0, 10.0, 0.0, 11.0, 11.0, 21.0, 0.0]",4371,94.0,10.0,0.0,11.0,11.0,21.0,0.0,3.0
33,54100,"[190.9, 10.0, 10.0, 10.0, 45.0, 15.0, 2.0]",5526,190.9,10.0,10.0,10.0,45.0,15.0,2.0,1.0
36,25775,"[166.1, 16.0, 6.0, 32.0, 19.0, 26.0, 3.0]",2420,166.1,16.0,6.0,32.0,19.0,26.0,3.0,3.0
40,90921,"[783.4, 46.0, 107.0, 56.0, 36.0, 21.0, 37.0]",9317,783.4,46.0,107.0,56.0,36.0,21.0,37.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
231564,283432,"[162.2, 16.0, 17.0, 9.0, 10.0, 30.0, 4.0]",23716,162.2,16.0,17.0,9.0,10.0,30.0,4.0,3.0
231566,96811,"[145.7, 13.0, 10.0, 2.0, 3.0, 27.0, 5.0]",9987,145.7,13.0,10.0,2.0,3.0,27.0,5.0,1.0
231572,455000,"[94.0, 7.0, 25.0, 4.0, 4.0, 3.0, 4.0]",29240,94.0,7.0,25.0,4.0,4.0,3.0,4.0,2.0
231600,208429,"[432.8, 32.0, 29.0, 39.0, 42.0, 39.0, 15.0]",19330,432.8,32.0,29.0,39.0,42.0,39.0,15.0,1.0


In [27]:
recipe_nutrition['n1'] = recipe_nutrition['fat'].apply(lambda x:1 if 15<=x<=30 else 0)
recipe_nutrition['n2'] = recipe_nutrition['sugar'].apply(lambda x:1 if x<10 else 0)
recipe_nutrition['n3'] = recipe_nutrition['sodium'].apply(lambda x:1 if x<83 else 0)
recipe_nutrition['n4'] = recipe_nutrition['protein'].apply(lambda x:1 if 10<=x<=15 else 0)
recipe_nutrition['n5'] = recipe_nutrition['saturated_fat'].apply(lambda x:1 if x<10 else 0)
recipe_nutrition['n6'] = recipe_nutrition['carbohydrates'].apply(lambda x:1 if 55<=x<=75 else 0)
recipe_nutrition = recipe_nutrition.sort_values(by='i').reset_index(drop=True)
numeric_columns = recipe_nutrition.iloc[:, -6:].values.tolist()
list_df = pd.DataFrame({'nut':numeric_columns})
concat = pd.concat([recipe_nutrition, list_df], axis = 1)
print(concat['nut'].value_counts())

multi_hot_health_dict = {}
for index, row in concat.iterrows():
    recipe_id = row['i']
    health_list = row['nut']
    multi_hot_health_dict[recipe_id] = health_list
rh_dict_mh = file_path + 'graph_edge/recipe_health_level_multi_hot_dict.pkl'

# 将字典保存到 pkl 文件中
with open(rh_dict_mh, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump(multi_hot_health_dict, f)
print("save health dict")
concat

[0, 0, 1, 0, 0, 0]    7658
[0, 0, 1, 0, 1, 0]    4868
[1, 0, 1, 0, 0, 0]    4365
[0, 1, 1, 0, 0, 0]    2957
[0, 1, 1, 0, 1, 0]    2216
[1, 1, 1, 0, 0, 0]    1681
[0, 0, 1, 1, 0, 0]    1143
[0, 0, 0, 0, 0, 0]    1036
[1, 0, 1, 1, 0, 0]     858
[0, 0, 1, 1, 1, 0]     661
[1, 0, 1, 0, 1, 0]     501
[0, 1, 1, 1, 1, 0]     350
[0, 1, 1, 1, 0, 0]     301
[1, 1, 1, 1, 0, 0]     286
[1, 1, 1, 0, 1, 0]     197
[0, 0, 0, 0, 1, 0]     128
[0, 0, 1, 0, 0, 1]     125
[1, 0, 0, 0, 0, 0]     105
[1, 0, 1, 1, 1, 0]     103
[0, 1, 0, 0, 0, 0]      97
[0, 0, 1, 0, 1, 1]      54
[0, 0, 0, 0, 0, 1]      44
[0, 0, 0, 1, 1, 0]      35
[0, 1, 0, 0, 1, 0]      30
[1, 1, 1, 1, 1, 0]      28
[1, 0, 0, 0, 1, 0]      26
[1, 1, 0, 0, 0, 0]      21
[1, 0, 1, 0, 0, 1]      17
[0, 0, 0, 1, 0, 0]      10
[1, 0, 0, 1, 1, 0]       4
[0, 1, 0, 1, 1, 0]       3
[0, 0, 1, 1, 1, 1]       3
[1, 1, 0, 0, 1, 0]       3
[1, 1, 1, 0, 0, 1]       3
[1, 0, 0, 0, 0, 1]       3
[0, 1, 0, 0, 0, 1]       3
[0, 1, 1, 0, 0, 1]       3
[

,id,nutrition,i,cal,fat,sugar,sodium,protein,saturated_fat,carbohydrates,health,n1,n2,n3,n4,n5,n6,nut
0,40,"[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]",0,311.1,0.0,308.0,0.0,0.0,0.0,27.0,2.0,0,0,1,0,1,0,"[0, 0, 1, 0, 1, 0]"
1,49,"[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",1,627.7,38.0,8.0,35.0,115.0,64.0,4.0,2.0,0,1,1,0,0,0,"[0, 1, 1, 0, 0, 0]"
2,58,"[280.1, 9.0, 36.0, 24.0, 59.0, 7.0, 7.0]",2,280.1,9.0,36.0,24.0,59.0,7.0,7.0,2.0,0,0,1,0,1,0,"[0, 0, 1, 0, 1, 0]"
3,66,"[772.0, 6.0, 657.0, 93.0, 13.0, 2.0, 63.0]",3,772.0,6.0,657.0,93.0,13.0,2.0,63.0,3.0,0,0,0,1,1,1,"[0, 0, 0, 1, 1, 1]"
4,142,"[224.8, 14.0, 87.0, 10.0, 7.0, 9.0, 11.0]",4,224.8,14.0,87.0,10.0,7.0,9.0,11.0,2.0,0,0,1,0,1,0,"[0, 0, 1, 0, 1, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29938,522871,"[63.7, 0.0, 20.0, 43.0, 8.0, 0.0, 3.0]",29938,63.7,0.0,20.0,43.0,8.0,0.0,3.0,2.0,0,0,1,0,1,0,"[0, 0, 1, 0, 1, 0]"
29939,522889,"[551.0, 71.0, 2.0, 42.0, 60.0, 127.0, 0.0]",29939,551.0,71.0,2.0,42.0,60.0,127.0,0.0,2.0,0,1,1,0,0,0,"[0, 1, 1, 0, 0, 0]"
29940,532736,"[139.0, 11.0, 56.0, 2.0, 5.0, 22.0, 5.0]",29940,139.0,11.0,56.0,2.0,5.0,22.0,5.0,1.0,0,0,1,0,0,0,"[0, 0, 1, 0, 0, 0]"
29941,532740,"[415.6, 27.0, 222.0, 4.0, 22.0, 30.0, 19.0]",29941,415.6,27.0,222.0,4.0,22.0,30.0,19.0,2.0,1,0,1,0,0,0,"[1, 0, 1, 0, 0, 0]"


In [28]:
recipe_nutrition['n1'] = recipe_nutrition['fat'].apply(lambda x:0 if 15<=x<=30 else -1)
recipe_nutrition['n2'] = recipe_nutrition['sugar'].apply(lambda x:1 if x<10 else -1)
recipe_nutrition['n3'] = recipe_nutrition['sodium'].apply(lambda x:2 if x<83 else -1)
recipe_nutrition['n4'] = recipe_nutrition['protein'].apply(lambda x:3 if 10<=x<=15 else -1)
recipe_nutrition['n5'] = recipe_nutrition['saturated_fat'].apply(lambda x:4 if x<10 else -1)
recipe_nutrition['n6'] = recipe_nutrition['carbohydrates'].apply(lambda x:5 if 55<=x<=75 else -1)
recipe_nutrition = recipe_nutrition.sort_values(by='i').reset_index(drop=True)
numeric_columns = recipe_nutrition.iloc[:, -6:].values.tolist()
list_df = pd.DataFrame({'nut':numeric_columns})
# concat = pd.concat([recipe_nutrition, list_df], axis = 1)
# print(concat['nut'].value_counts())
# concat

import scipy.sparse as sp
import numpy as np

item_nut_mat = sp.dok_matrix(
        (len(item_to_idx), 6), dtype=np.float32
    )

for item, ingres in tqdm(enumerate(numeric_columns)):
    ingre = [i for i in ingres if i!=-1]
    item_nut_mat[item, ingre] = 1

item_nut_mat_csr = item_nut_mat.tocsr()
item_item_mat_ = item_nut_mat_csr @ item_nut_mat_csr.T

item_item_mat_.setdiag(0)
item_item_mat_.data[item_item_mat_.data<=3]=0
item_item_mat_.eliminate_zeros()

print(item_item_mat_.nnz)

graph_data = []
for i, row in enumerate(tqdm(item_item_mat_, total=item_item_mat_.shape[0])):
    for j, r in zip(row.indices, row.data):
        graph_data.append((i, j, r))
print("edge number of graph", len(graph_data))

rr_health_array = np.array(graph_data).astype(np.int_)
print(rr_health_array.shape)

rr_health_path = file_path + 'graph_edge/rr_health_graph.txt'
with open(rr_health_path, 'w') as f:
    np.savetxt(f, rr_health_array, fmt='%d')
print("NumPy array saved as 'rr_health_graph.txt'")

with open(rr_health_path, 'r') as f:
    ab = np.loadtxt(f, dtype=np.int_)
print(f"testing for shape, rr_health_graph is {ab.shape}")



29943it [00:02, 14037.35it/s]
/home/miniconda3/envs/zyx2509/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


306232


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29943/29943 [00:01<00:00, 28480.83it/s]


edge number of graph 306232
(306232, 3)
NumPy array saved as 'rr_health_graph.txt'
testing for shape, rr_health_graph is (306232, 3)


In [29]:
health_dict = {}
for index, row in recipe_nutrition.iterrows():
    recipe_id = row['i']
    health_amount = row['health']
    health_dict[recipe_id] = int(health_amount)
rh_dict = file_path + 'graph_edge/recipe_health_level_dict.pkl'

# 将字典保存到 pkl 文件中
with open(rh_dict, 'wb') as f:  # 注意要以二进制写入模式打开文件
    pickle.dump(health_dict, f)
print("save health dict")

# recipe-health graph edge
rh_list = []
for key, value in health_dict.items():
    rh_list.append([key, value])
rh_array = np.array(rh_list).astype(np.int_)
print(rh_array.shape)

rh_path = file_path + 'graph_edge/rh_graph.txt'
with open(rh_path, 'w') as f:
    np.savetxt(f, rh_array, fmt='%d')
print("NumPy array saved as 'rh_graph.txt'")

with open(rh_path, 'r') as f:
    ab = np.loadtxt(f, dtype=np.int_)
print(f"testing for shape, rh_graph is {ab.shape}")


max_index = recipe_nutrition['protein'].idxmax()  # 找到最大值所在的索引
max_row = recipe_nutrition.loc[max_index]  # 获取最大值所在的整行数据
max_row

save health dict
(29943, 2)
NumPy array saved as 'rh_graph.txt'
testing for shape, rh_graph is (29943, 2)


id                                                           93481
nutrition        [38662.3, 4331.0, 77.0, 809.0, 6552.0, 4969.0,...
i                                                             9596
cal                                                        38662.3
fat                                                         4331.0
sugar                                                         77.0
sodium                                                       809.0
protein                                                     6552.0
saturated_fat                                               4969.0
carbohydrates                                                  8.0
health                                                         0.0
n1                                                              -1
n2                                                              -1
n3                                                              -1
n4                                                            